In [ ]:
using ArgCheck
using DataFrames
using Distributions
using HDPHMM
using Glob
using JSON
using ModelComparison
using ProgressMeter

In [ ]:
# TODO: Move this (and HDPHMM.jl/src/io.jl HMM(...))
# to HMMBase ? With generic parse code for distributions.
function HMMBase.HMM(D::Type{<:Distribution}, d::Dict)
    a = Vector{Float64}(d["a"])
    A = Matrix{Float64}(hcat(d["A"]...))
    B = map(D, d["B"])
    HMM(a, A, B)
end

struct DataModel
    data::Vector{Float64}
    dpmm::MixtureModel
    hdphmm::HMM
    hdpghmm::HMM
end

function DataModel(o::Dict)
    data = Vector{Float64}(o["data"])
    dpmm = MixtureModel(Normal, o["DPMM"])
    hdphmm = HMM(MixtureModel, Normal, o["HDPHMM"])
    hdpghmm = HMM(MixtureModel, Normal, o["HDPGHMM"])
    DataModel(data, dpmm, hdphmm, hdpghmm)
end

function JSON.parsefile(::Type{DataModel}, filename)
    DataModel(parsefile(filename))
end

### Main

In [ ]:
function calibrate_hyptest(h0::Union{Distribution,HMM}, h1::Union{Distribution,HMM}, T::Integer, N::Integer)
    @argcheck T > 0
    @argcheck N > 0

    ratios = zeros(N)

    for n in eachindex(ratios)
        # (1) Generate time series of length T according to `hmm` (H0)
        # (2) Compute the log of the likelihood ratio hmm / hdphmm
        y = rand(h0, T)
        ratios[n] = (loglikelihood(h0, y)/length(y)) - (loglikelihood(h1, y)/length(y))
    end

    αs = range(0.0, 1.0, step = 0.05)
    ks = zeros(size(αs))

    for (i, α) in enumerate(αs)
        if α == 1.0
            ks[i] = Inf
        elseif α == 0.0
            ks[i] = -Inf
        else
            # Find k such that sum(ratios .<= k) = N*α
            x = range(extrema(ratios)..., length = 2000)
            r = [abs(sum(ratios .<= k) - N*α) for k in x]
            ks[i] = x[findmin(r)[2]]
        end
    end

    αs, ks
end

In [ ]:
files = glob("*.models.json", "../data/ping_v4_1580511600_1581116400_noself_pairs/");

In [ ]:
tests = DataFrame([[], [], [], [], [], []], [:file, :h0, :alpha, :k, :ratio, :result])
h1 = :hdphmm
N = 200

@showprogress for file in files
    try
        model = parsefile(DataModel, file)
        for h0 in [:dpmm, :hdpghmm]
            m0 = getfield(model, h0)
            m1 = getfield(model, h1)
            αs, ks = calibrate_hyptest(m0, m1, length(model.data), N)
            ratio = (loglikelihood(m0, model.data)/length(model.data)) - (loglikelihood(m1, model.data)/length(model.data))
            for (α, k) in zip(αs, ks)
                push!(tests, [file, h0, α, k, ratio, ratio <= k])
            end
        end
    catch e
        (e isa InterruptException) && (break)
        @warn e
    end
end

In [ ]:
for h0 in unique(tests.h0), α in unique(tests.alpha)
    df_ = tests[(tests.h0 .== h0) .& (tests.alpha .== α), :]
    accepted = sum(df_.result) / size(df_.result, 1)
    println("H0 = $h0, alpha = $α, accepted = $accepted")
end

### Plots

(I was in a hurry to generate the plots, so instead of re-running the simulation I just parsed the text output from the previous cells. A much better way would be to store the results directly :-))

In [ ]:
using PyPlot
using ThesisTools

In [ ]:
results_dpmm = """H0 = dpmm, alpha = 0.0, accepted = 0.0
H0 = dpmm, alpha = 0.05, accepted = 0.9461663947797716
H0 = dpmm, alpha = 0.1, accepted = 0.9477977161500816
H0 = dpmm, alpha = 0.15, accepted = 0.9494290375203915
H0 = dpmm, alpha = 0.2, accepted = 0.9510603588907015
H0 = dpmm, alpha = 0.25, accepted = 0.9510603588907015
H0 = dpmm, alpha = 0.3, accepted = 0.9510603588907015
H0 = dpmm, alpha = 0.35, accepted = 0.9510603588907015
H0 = dpmm, alpha = 0.4, accepted = 0.9510603588907015
H0 = dpmm, alpha = 0.45, accepted = 0.9526916802610114
H0 = dpmm, alpha = 0.5, accepted = 0.9526916802610114
H0 = dpmm, alpha = 0.55, accepted = 0.9543230016313213
H0 = dpmm, alpha = 0.6, accepted = 0.9608482871125612
H0 = dpmm, alpha = 0.65, accepted = 0.965742251223491
H0 = dpmm, alpha = 0.7, accepted = 0.9706362153344209
H0 = dpmm, alpha = 0.75, accepted = 0.9771615008156607
H0 = dpmm, alpha = 0.8, accepted = 0.9853181076672104
H0 = dpmm, alpha = 0.85, accepted = 0.9934747145187602
H0 = dpmm, alpha = 0.9, accepted = 0.9934747145187602
H0 = dpmm, alpha = 0.95, accepted = 0.9967373572593801
H0 = dpmm, alpha = 1.0, accepted = 1.0"""

results_hdpghmm = """H0 = hdpghmm, alpha = 0.0, accepted = 0.0
H0 = hdpghmm, alpha = 0.05, accepted = 0.653910149750416
H0 = hdpghmm, alpha = 0.1, accepted = 0.697171381031614
H0 = hdpghmm, alpha = 0.15, accepted = 0.7271214642262895
H0 = hdpghmm, alpha = 0.2, accepted = 0.7504159733777038
H0 = hdpghmm, alpha = 0.25, accepted = 0.7770382695507487
H0 = hdpghmm, alpha = 0.3, accepted = 0.7870216306156406
H0 = hdpghmm, alpha = 0.35, accepted = 0.800332778702163
H0 = hdpghmm, alpha = 0.4, accepted = 0.8186356073211315
H0 = hdpghmm, alpha = 0.45, accepted = 0.8236272878535774
H0 = hdpghmm, alpha = 0.5, accepted = 0.8452579034941764
H0 = hdpghmm, alpha = 0.55, accepted = 0.8569051580698835
H0 = hdpghmm, alpha = 0.6, accepted = 0.8618968386023295
H0 = hdpghmm, alpha = 0.65, accepted = 0.8685524126455907
H0 = hdpghmm, alpha = 0.7, accepted = 0.8768718801996672
H0 = hdpghmm, alpha = 0.75, accepted = 0.8885191347753744
H0 = hdpghmm, alpha = 0.8, accepted = 0.8968386023294509
H0 = hdpghmm, alpha = 0.85, accepted = 0.9018302828618968
H0 = hdpghmm, alpha = 0.9, accepted = 0.9101497504159733
H0 = hdpghmm, alpha = 0.95, accepted = 0.9217970049916805
H0 = hdpghmm, alpha = 1.0, accepted = 1.0""";

In [ ]:
αs = 0:0.05:1.0
dpmm = map(x -> parse(Float64, split(x, "accepted = ")[end]) * 100, split(results_dpmm, "\n"))
hdpghmm = map(x -> parse(Float64, split(x, "accepted = ")[end]) * 100, split(results_hdpghmm, "\n"));

In [ ]:
fig, ax = subplots()
ax.scatter(quant_tho, quant_obs)
ax.plot([0, 5], [0, 5], label = L"$y = x$")
ax.set(xlim = (0, 5), ylim = (0, 5), xlabel = "Quantiles théoriques Exp(1)", ylabel = "Quantiles observés")
ax.grid()
save_thesis("atlas_durations_qq")

In [ ]:
fig, ax = subplots()
ax.plot(αs, dpmm, marker="x", label = "\$H_0\$ : DPMM, \$H_1\$ : HDP-HMM-G")
ax.plot(αs, hdpghmm, marker="x", label = "\$H_0\$ : HDP-HMM-G, \$H_1\$ : HDP-HMM-D")
ax.grid()
ax.legend(loc = "lower right")
ax.set(xlabel = "Probabilité de fausse alarme \$\\alpha\$", ylabel = "Taux de rejet de \$H_0\$ (%)")
save_thesis("likelihood_ratio_test")